<a href="https://colab.research.google.com/github/ahmadabousalem/sentinel-threat-map/blob/main/risk%20colour%20map3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:

!pip install feedparser folium pandas --quiet

import feedparser
import folium
import pandas as pd
import html
from IPython.display import IFrame, HTML, display
from datetime import datetime
import time

# ------------------ CONFIG ------------------
cities = {
    "Gaza": {"lat": 31.5017, "lon": 34.4668},
    "Beirut": {"lat": 33.8938, "lon": 35.5018},
    "Damascus": {"lat": 33.5138, "lon": 36.2765},
    "Northern Israel Border": {"lat": 33.27, "lon": 35.57}
}

# RSS feeds (free/open). If some are slow, remove while testing.
rss_feeds = [
    "https://www.aljazeera.com/xml/rss/all.xml",
    "http://feeds.bbci.co.uk/news/world/middle_east/rss.xml",
    "https://rss.dw.com/rdf/rss-en-all",
    "http://rss.cnn.com/rss/edition_world.rss",
    "http://feeds.reuters.com/Reuters/worldNews"
]

# Simple risk keywords
risk_keywords = [
    'attack', 'explosion', 'war', 'strike', 'missile', 'death',
    'protest', 'conflict', 'rocket', 'terror', 'bomb', 'hostage',
    'shooting', 'shelling', 'air raid'
]

# ------------------ HELPERS ------------------
def safe_parse(url):
    try:
        parsed = feedparser.parse(url)
        # basic check
        if parsed.bozo:
            # bozo flag means parse problem; still try to return entries if any
            return parsed
        return parsed
    except Exception as e:
        print(f"[WARN] Feed parse failed for {url}: {e}")
        return None

def fetch_risk_data(city_names, feeds, keywords, max_headlines=5):
    """
    Returns:
      counts: dict city -> integer score (sum keyword hits)
      headlines: dict city -> list of (pub, title, link)
    """
    counts = {c: 0 for c in city_names}
    headlines = {c: [] for c in city_names}
    for url in feeds:
        parsed = safe_parse(url)
        if not parsed or not hasattr(parsed, "entries"):
            continue
        for entry in parsed.entries:
            title = entry.get("title", "") or ""
            summary = entry.get("summary", "") or entry.get("description", "") or ""
            text = (title + " " + summary).lower()
            pub = entry.get("published", entry.get("updated", "N/A"))
            link = entry.get("link", "")
            for city in city_names:
                if city.lower() in text:
                    # count keyword matches (if none match, count as 1)
                    kcount = sum(1 for k in keywords if k in text)
                    counts[city] += (kcount if kcount>0 else 1)
                    # add headline (avoid duplicates)
                    entry_label = f"{pub} | {title}"
                    if entry_label not in [h[0] for h in headlines[city]]:
                        headlines[city].append((entry_label, link))
                        # keep only newest N headlines
                        headlines[city] = headlines[city][:max_headlines]
        # polite pause to avoid hammering feeds
        time.sleep(0.3)
    return counts, headlines

def classify_risk(score):
    if score >= 8:
        return "HIGH"
    elif score >= 4:
        return "MEDIUM"
    elif score > 0:
        return "LOW"
    else:
        return "NONE"

# ------------------ RUN FETCH ------------------
print("Fetching RSS feeds (may take 10-30s)...")
counts, headlines = fetch_risk_data(list(cities.keys()), rss_feeds, risk_keywords, max_headlines=5)
print("Done.\n")

# Make DataFrame for display
df = pd.DataFrame([{"city": c, "score": counts.get(c,0), "risk": classify_risk(counts.get(c,0))} for c in cities.keys()])
df = df.sort_values("score", ascending=False).reset_index(drop=True)
display(df)

# ------------------ BUILD MAP ------------------
m = folium.Map(location=[33.5, 35.4], zoom_start=7, tiles='CartoDB positron')

color_map = {"HIGH":"darkred", "MEDIUM":"orange", "LOW":"blue", "NONE":"green"}

for _, row in df.iterrows():
    city = row["city"]
    score = int(row["score"])
    level = row["risk"]
    coords = (cities[city]["lat"], cities[city]["lon"])
    color = color_map[level]
    radius = 6 + score * 2

    # Build popup HTML safely with links
    popup_html = f"<div style='max-width:350px'><b>{html.escape(city)}</b><br><b>Risk:</b> {level} (score {score})<br><hr>"
    if headlines.get(city):
        popup_html += "<b>Recent headlines:</b><br>"
        for h, link in headlines[city]:
            safe_h = html.escape(h)
            if link:
                popup_html += f"<a href='{link}' target='_blank'>{safe_h}</a><br>"
            else:
                popup_html += f"{safe_h}<br>"
    else:
        popup_html += "No recent headlines found.<br>"
    popup_html += "</div>"

    folium.CircleMarker(
        location=coords,
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=400),
        tooltip=f"{city}: {level} ({score})"
    ).add_to(m)

map_file = "sentinel_risk_map.html"
m.save(map_file)
print(f"Map saved to: {map_file}")

# Display map inline (two methods for robustness)
try:
    display(HTML(m._repr_html_()))
except Exception:
    display(IFrame(map_file, width="100%", height=600))
# ------------------ End of cell ------------------



Fetching RSS feeds (may take 10-30s)...
Done.



,city,score,risk
0,Gaza,47,HIGH
1,Beirut,1,LOW
2,Damascus,0,NONE
3,Northern Israel Border,0,NONE


Map saved to: sentinel_risk_map.html
